In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import time, threading, re, os
import warnings
warnings.filterwarnings("ignore")

In [2]:
def config():
    if not os.path.exists('zhh_configs.txt') or not open('zhh_configs.txt', encoding='utf-8').readline(): #无记录或记录为空
        info_list=[]
        username=input('请输入智慧树用户名（手机号）：')
        info_list.append(username+'\n')
        password=input('请输入密码：')
        info_list.append(password+'\n')
        url=input('请输入课程视频页链接：')
        info_list.append(url+'\n')
        audio=input('是否播放声音？`audio/mute`:')
        info_list.append(audio+'\n')
        exp=input('是否显示浏览器窗口？`explore/headless`：')
        info_list.append(exp+'\n')
        open('zhh_configs.txt', mode='w', encoding='utf-8').writelines(info_list)
    else:
        info=open('zhh_configs.txt', encoding='utf-8').readlines()
        username=info[0]
        password=info[1]
        url=info[2]
        audio=info[3]
        exp=info[4]
        
    return [username, password, url, audio, exp]

In [3]:
def login(config):
    username=config[0]
    password=config[1]
    url=config[2]
    audio=config[3]
    exp=config[4]
    if re.match(r'https://studyh5',url):
        url_type='lesson'
    elif re.match(r'https://lc.zhihuishu.com/live',url):
        url_type='live'
    else:
        url_type='invaild type'
    
    chrome_options = webdriver.ChromeOptions()
    if audio=='mute':
        chrome_options.add_argument("--mute-audio")
    if exp=='headless':
        chrome_options.add_argument("--headless")
    dr = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
    dr.get(url)
    time.sleep(5)
    dr.find_element_by_name('username').send_keys(username)
    dr.find_element_by_name('password').send_keys(password)
    dr.find_element_by_class_name('wall-sub-btn').click()
    return [dr, url_type]

In [4]:
class Lesson():
    def __init__(self, dr):
        self.dr=dr
        self.skip_iknow()
        time.sleep(1)
        #首次设置流畅
        self.dr.execute_script('document.querySelector("#vjs_container > div.controlsBar > div.definiBox > div > b.line1bq.switchLine").click()')
        title=self.dr.find_element_by_xpath('//*[@id="lessonOrder"]').get_attribute('textContent')
        print("正在播放课程："+title)
    
    def skip_iknow(self): #关闭我知道了
        try:
            time.sleep(5)
            WebDriverWait(self.dr, 10, 0.5).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[1]/div/div[6]/div/div[1]/button')))
            self.dr.find_element_by_xpath('/html/body/div[1]/div/div[6]/div/div[1]/button').click()
        except:
            pass
        try:
            time.sleep(2)
            WebDriverWait(self.dr, 10, 0.5).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[1]/div/div[7]/div[2]/div[1]/i')))
            self.dr.find_element_by_xpath('/html/body/div[1]/div/div[7]/div[2]/div[1]/i').click()
        except:
            pass
        
    def is_exist(self):
        while True:
            try:
                self.dr.find_element_by_class_name('topic-item').click()#选第一个选项
                self.dr.find_element_by_xpath('/html/body/div[1]/div/div[7]/div/div[3]/span/div').click()#关闭答题窗口
                self.dr.execute_script('document.querySelector("#playButton").click()')#点一下暂停/播放按钮
            except:
                pass
            time.sleep(5)#每隔5秒检测弹窗
            
    def close_timeup(self):
        Locker=False
        while True:
            try:
                self.dr.execute_script('document.querySelector("#app > div > div.el-dialog__wrapper.dialog-look-habit > div > div.el-dialog__header > button > i").click()')
                self.dr.execute_script('document.querySelector("#playButton").click()')#点一下暂停/播放按钮
                Locker=True
            except:
                pass
            if Locker:
                print('\n')
                print('今日习惯时长已满',end='\n')
            time.sleep(60)#每隔60秒检测弹窗   
    
    def show_progress(self):
        while True:
            time.sleep(1)#每间隔1秒检测视频是否播放完
            try:
                #该视频的总时间
                total_time=self.dr.find_element_by_xpath('//*[@id="vjs_container"]/div[10]/div[4]/span[2]').get_attribute('textContent')
                #print(current_time,total_time)            
                #获取当前播放的进度
                current_time=self.dr.find_element_by_xpath('//*[@id="vjs_container"]/div[10]/div[4]/span[1]').get_attribute('textContent')
                print("进度:{0}/{1}".format(current_time,total_time), end="\r")

                if current_time[3:5]==total_time[3:5] and int(current_time[6:])>=(int(total_time[6:])-2):
                    #当前视频播放结束，点击下一节  
                    self.dr.execute_script("document.querySelector('#nextBtn').click()")
                    print('\n')
                    print('自动播放下一节',end='\n')
                    
                    time.sleep(6)
                    #title=dr.find_element_by_xpath('//*[@id="lessonOrder"]').get_attribute('textContent')
                    print("正在播放："+ self.dr.find_element_by_xpath('//*[@id="lessonOrder"]').get_attribute('textContent'))
                    #流畅
                    self.dr.execute_script('document.querySelector("#vjs_container > div.controlsBar > div.definiBox > div > b.line1bq.switchLine").click()')
                    #time.sleep(2)
                    #1.5倍关闭状态，有bug
                    #dr.execute_script('document.querySelector("#vjs_container > div.controlsBar > div.speedBox > div > div.speedTab.speedTab15").click()')
                    current_time = '00:00:00'
                    total_time = '00:00:10'

            except:
                current_time = '00:00'
                total_time = '00:10'
                
    def close_web(self):#定时30分钟后关闭
        for timer in range(10):#每隔3分钟打印进度
            time.sleep(180)
            print('\n')
            print('视频已播放'+str((timer+1)*3)+'分钟',end='\n')
            if timer==8:
                print('\n')
                print('将于3分钟后自动关闭',end='\n')
        self.dr.execute_script('document.querySelector("#playButton").click()') #暂停
        self.dr.quit() #退出
        quit()

In [5]:
class Live():
    def __init__(self, dr):
        self.dr=dr
        self.dr.implicitly_wait(10)
        time.sleep(2)
        WebDriverWait(self.dr, 10, 0.5).until_not(EC.presence_of_element_located((By.XPATH,'//*[@id="popbox_title"]')))
        if not Live.is_element_present(self.dr, By.XPATH, '//*[@id="popbox_title"]'):
            #首次设置流畅
            self.dr.execute_script('document.querySelector("#vjs_forFollowBackDiv > div.controlsBar > div.definiBox > div > b.line1bq.switchLine").click()')
            title=self.dr.find_element_by_xpath('//*[@id="wh_live_name"]').get_attribute('textContent')
            print("正在播放回放："+title)

    def is_element_present(driver, by, value): #判断网页元素是否存在
        try:
            element = driver.find_element(by=by, value=value)
        except NoSuchElementException as e:
            return False
        return True

    def str2sec(time_str):
        h, m, s = time_str.strip().split(':') 
        return int(h)*3600 + int(m)*60 + int(s) 

    def show_progress(self):
        progress=self.dr.find_element_by_xpath('//*[@id="container"]/div[1]/div/div[2]/div[1]/p[2]/span').get_attribute('textContent')
        print("目前签到进度："+progress+"%")
        while True:
            time.sleep(1)#每间隔1秒检测视频是否播放完
            try:
            #该视频的总时间
                total_time=self.dr.find_element_by_xpath('//*[@id="vjs_forFollowBackDiv"]/div[10]/div[4]/span[2]').get_attribute('textContent')      
                #获取当前播放的进度
                current_time=self.dr.find_element_by_xpath('//*[@id="vjs_forFollowBackDiv"]/div[10]/div[4]/span[1]').get_attribute('textContent')                       
                print("进度:{0}/{1}".format(current_time,total_time), end="\r")
                
                if current_time!='00:00:00' and total_time!='00:00:00':
                    if current_time[3:5]==total_time[3:5] and int(current_time[6:])>=(int(total_time[6:])-2):
                        print('\n')
                        print('已完成本次直播',end='\n')
                        self.dr.quit() #退出
                        quit()
                    
            except:
                current_time = '00:00'
                total_time = '00:10'
    
    
    def play_time(self):
        while True:
            time.sleep(0.5)
            try:
                progress=self.dr.find_element_by_xpath('//*[@id="container"]/div[1]/div/div[2]/div[1]/p[2]/span').get_attribute('textContent')

                total_time=self.dr.find_element_by_xpath('//*[@id="vjs_forFollowBackDiv"]/div[10]/div[4]/span[2]').get_attribute('textContent')  
                total_sec=Live.str2sec(total_time)

                current_time=self.dr.find_element_by_xpath('//*[@id="vjs_forFollowBackDiv"]/div[10]/div[4]/span[1]').get_attribute('textContent')                      
                current_sec=Live.str2sec(current_time)

                sign_sec=int(total_sec*int(progress)/100) #最后签到的时间
                iterate=int((sign_sec-current_sec)/60)
                for i in range(iterate):
                    self.dr.execute_script("document.getElementsByTagName('video')[0].currentTime = document.getElementsByTagName('video')[0].currentTime + 60")
                    time.sleep(1)
                    i+=1
                    if i==iterate:
                        print('\n')
                        print('已到达上次观看位置', end='\n')
                        break
            except:
                pass

In [6]:
def main():
    configs=config()
    configs=[i.strip('\n') for i in configs]
    logins=login(configs)
    driver=logins[0]
    class_type=logins[1]
    
    threads=[]
    if class_type=='lesson':
        Zhihuishu=Lesson(driver)
        threads.append(threading.Thread(target=Zhihuishu.is_exist))
        threads.append(threading.Thread(target=Zhihuishu.show_progress))
        threads.append(threading.Thread(target=Zhihuishu.close_web))
        threads.append(threading.Thread(target=Zhihuishu.close_timeup))
        
    elif class_type=='live':
        Zhihuishu=Live(driver)
        threads.append(threading.Thread(target=Zhihuishu.show_progress))
        threads.append(threading.Thread(target=Zhihuishu.play_time))
    
    else:
        raise Exception('请检查输入的链接，需包含http头')
    
    for thr in threads:
        thr.setDaemon(True)
        thr.start()
        
    for thr in threads:
        if thr.isAlive: #阻止主线程直接结束
            thr.join()

In [ ]:
if __name__=='__main__':
    main()

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\92859\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache
